In [54]:
import functools

import numpy as np
import tensorflow as tf

In [55]:
print(tf.__version__)

2.8.0


In [56]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [57]:
# # Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [58]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [59]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [60]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [61]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [62]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'male']
age                 : [31.  2. 14. 28. 28.]
n_siblings_spouses  : [0 4 1 0 0]
parch               : [0 1 0 0 0]
fare                : [10.5   29.125 11.242  7.75   9.5  ]
class               : [b'Second' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'F' b'unknown']
embark_town         : [b'Southampton' b'Queenstown' b'Cherbourg' b'Queenstown' b'Southampton']
alone               : [b'y' b'n' b'n' b'y' b'y']


In [63]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'female' b'female' b'male']
age                 : [23. 37. 47. 28. 28.]
n_siblings_spouses  : [0 1 1 0 0]
parch               : [0 0 0 2 1]
fare                : [13.   53.1  14.5   7.75 33.  ]
class               : [b'Second' b'First' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'C' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Queenstown' b'Southampton']
alone               : [b'y' b'n' b'n' b'n' b'n']


In [64]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [49.  47.  32.5 28.  40. ]
n_siblings_spouses  : [0 1 0 1 1]
class               : [b'First' b'First' b'Second' b'Third' b'Second']
deck                : [b'D' b'D' b'E' b'unknown' b'unknown']
alone               : [b'y' b'n' b'y' b'n' b'n']


In [65]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [32. 19. 29. 24. 35.]
n_siblings_spouses  : [0. 0. 1. 2. 0.]
parch               : [0. 0. 1. 0. 0.]
fare                : [ 7.75   7.896 10.462 24.15   8.05 ]


In [66]:
example_batch, labels_batch = next(iter(temp_dataset))

In [67]:
# 다음은 모든 열을 묶는 간단한 함수입니다.
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [68]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[ 35.      0.      0.     21.   ]
 [  0.75    2.      1.     19.258]
 [ 30.      0.      0.      7.896]
 [ 24.      0.      1.    247.521]
 [ 47.      0.      0.     34.021]]

[1 1 0 0 0]


In [69]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [28. 47. 36. 29.  1.]
n_siblings_spouses  : [0 0 1 1 4]
parch               : [0 0 1 0 1]
fare                : [ 7.896 25.587 24.15  66.6   39.688]
class               : [b'Third' b'First' b'Third' b'First' b'Third']
deck                : [b'unknown' b'E' b'unknown' b'C' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'n' b'n' b'n']


In [70]:
example_batch, labels_batch = next(iter(temp_dataset))

In [71]:
# 따라서 숫자 특성 목록을 선택하고 단일 열로 묶는 보다 일반적인 전처리기를 정의합니다.
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [72]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [73]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'female']
class               : [b'Third' b'Third' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'y']
numeric             : [[28.     0.     0.     8.113]
 [20.     0.     0.     7.229]
 [23.     0.     0.     7.925]
 [22.     0.     0.     7.25 ]
 [28.     0.     0.    13.   ]]


In [74]:
example_batch, labels_batch = next(iter(packed_train_data))

In [75]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [76]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [77]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [78]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f9cea9448c0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [79]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[39.   ,  1.   ,  5.   , 31.275],
       [40.   ,  0.   ,  0.   , 27.721],
       [ 0.75 ,  2.   ,  1.   , 19.258],
       [24.   ,  0.   ,  0.   ,  7.142],
       [28.   ,  0.   ,  0.   ,  7.896]], dtype=float32)>

In [80]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 0.749,  0.395,  5.827, -0.057],
       [ 0.829, -0.474, -0.479, -0.122],
       [-2.308,  1.264,  0.782, -0.277],
       [-0.45 , -0.474, -0.479, -0.499],
       [-0.13 , -0.474, -0.479, -0.485]], dtype=float32)

범주형 데이터

In [81]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [82]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [83]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [84]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


결합된 전처리 레이어

In [85]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [86]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.749  0.395
  5.827 -0.057  1.     0.   ]


In [87]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [88]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [89]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 3s 8ms/step - loss: 0.5062 - accuracy: 0.7337
Epoch 2/20
126/126 [==============================] - 1s 4ms/step - loss: 0.4213 - accuracy: 0.8118
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4057 - accuracy: 0.8230
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3912 - accuracy: 0.8373
Epoch 5/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3744 - accuracy: 0.8325
Epoch 6/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3713 - accuracy: 0.8421
Epoch 7/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3660 - accuracy: 0.8421
Epoch 8/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3555 - accuracy: 0.8485
Epoch 9/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3468 - accuracy: 0.8501
Epoch 10/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3413 - accuracy: 0.8612

In [90]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 0s 2ms/step - loss: 0.4997 - accuracy: 0.8258


Test Loss 0.49971118569374084, Test Accuracy 0.8257575631141663


In [91]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 100.00%  | Actual outcome:  SURVIVED
Predicted survival: 16.64%  | Actual outcome:  DIED
Predicted survival: 97.02%  | Actual outcome:  SURVIVED
Predicted survival: 62.72%  | Actual outcome:  DIED
Predicted survival: 98.60%  | Actual outcome:  SURVIVED
